# Thư viện

In [1]:
!pip install -q flwr[simulation]  pandas matplotlib scikit-learn torch

In [2]:
from collections import OrderedDict
from typing import List, Tuple
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import pickle

import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Metrics, Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
from flwr.server.client_proxy import ClientProxy
from flwr.common import Parameters, Scalar, FitRes, parameters_to_ndarrays
from typing import Optional, Union

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")


Training on cpu
Flower 1.13.0 / PyTorch 2.5.1


In [3]:
# Cấu hình thiết bị (CUDA hoặc CPU)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset

In [4]:
# Đường dẫn đến tập dữ liệu
DATA_PATH = r"..\data\SMSSpamCollection"

# Đọc dữ liệu
data = pd.read_csv(DATA_PATH, sep='\t', header=None, names=["label", "text"])

# Chuyển đổi nhãn 'ham' và 'spam' thành 0 và 1
data["label"] = data["label"].map({"ham": 0, "spam": 1})


In [5]:
BATCH_SIZE = 16
NUM_CLIENTS = 10

## Xử lý dữ liệu

### Data Poisoning Attack

Nếu partition_id là 1, 3 hoặc 5, dữ liệu của client sẽ bị nhiễu (poisoning attack). Chi tiết tấn công như sau: 

2. **Text Corruption**:
   - Thêm từ ngẫu nhiên từ danh sách ["spam", "fake", "malicious", "attack"] vào nội dung tin nhắn để làm "bẩn" dữ liệu văn bản.

In [6]:
class SMSDataset(Dataset):
    def __init__(self, texts, labels, vectorizer):
        self.texts = texts
        self.labels = labels
        self.vectorizer = vectorizer

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text_vector = self.vectorizer.transform([self.texts[idx]]).toarray().squeeze()
        label = self.labels[idx]
        return torch.tensor(text_vector, dtype=torch.float32), torch.tensor(label, dtype=torch.long)
    
def add_noise(text):
    """Thêm nhiễu vào văn bản bằng cách thay thế từ khóa hoặc chỉnh sửa nội dung."""
    keywords_to_replace = {"free": "offer", "win": "get", "urgent": "important", "claim": "request"}
    words = text.split()
    noisy_words = [keywords_to_replace.get(word, word) for word in words]
    return " ".join(noisy_words)

def load_datasets(partition_id: int):
    print(f"Loading dataset for partition ID: {partition_id}")

    # Load raw data
    data = pd.read_csv(DATA_PATH, sep="\t", header=None, names=["label", "text"])
    data["label"] = data["label"].map({"ham": 0, "spam": 1})

    # Vectorize text data
    vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
    vectorizer.fit(data["text"])

    # Shuffle data and reset index
    data = data.sample(frac=1, random_state=42).reset_index(drop=True)

    # Split into NUM_CLIENTS partitions
    indices = np.arange(len(data))
    split_indices = np.array_split(indices, NUM_CLIENTS)
    partition_indices = split_indices[partition_id]
    partition_data = data.iloc[partition_indices]

    # Poisoning attack
    if partition_id in [1, 3, 5]:

        print("Client 1,3,5 is applying poisoning attack.")

        # Poison 90% of the data
        num_poisoned = int(0.9 * len(partition_data))  
        poisoned_indices = np.random.choice(len(partition_data), num_poisoned, replace=False)

        # Adjust poisoned indices based on the partitioning (shuffled indices)
        poisoned_indices = partition_indices[poisoned_indices]

        # Flip labels 
        partition_data.loc[poisoned_indices, "label"] = partition_data.loc[poisoned_indices, "label"].apply(lambda x: 1 - x)

       # Add realistic noise to the text
        for idx in poisoned_indices:
            original_text = partition_data.loc[idx, "text"]
            partition_data.loc[idx, "text"] = add_noise(original_text)

    # Train/val split
    train_data, val_data = train_test_split(partition_data, test_size=0.1, random_state=42)
    train_dataset = SMSDataset(train_data["text"].tolist(), train_data["label"].tolist(), vectorizer)
    val_dataset = SMSDataset(val_data["text"].tolist(), val_data["label"].tolist(), vectorizer)

    # Dataloaders
    trainloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    # Test set (for client 5, use its own partition data for testing, for others use last partition)
    if partition_id in [1, 3, 5]:
        test_data = partition_data  # Use the same poisoned data for testing for client 5
    else:
        test_data = data.iloc[split_indices[-1]]  # Test data from last partition for all other clients

    test_dataset = SMSDataset(test_data["text"].tolist(), test_data["label"].tolist(), vectorizer)
    testloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

    print(f"Partition {partition_id}: Train {len(train_data)}, Val {len(val_data)}, Test {len(test_data)}")
    return trainloader, valloader, testloader



In [7]:
data.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


# Train model

In [8]:
class Net(nn.Module):
    def __init__(self, input_dim: int):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)  # Fully connected layer
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)  # 2 output classes (ham, spam)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [9]:
# Lấy trainloader từ partition đầu tiên
trainloader, valloader, testloader = load_datasets(partition_id=0)

# Số chiều của đầu vào từ vectorizer
input_dim = 5000  # (đã được đặt trong load_datasets max_features=5000)

# Khởi tạo mô hình
net = Net(input_dim).to(DEVICE)
print(net)


Loading dataset for partition ID: 0
Partition 0: Train 502, Val 56, Test 557
Net(
  (fc1): Linear(in_features=5000, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)


In [10]:
def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        epoch_loss = 0.0
        correct, total = 0, 0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss.item()
            total += labels.size(0)
            correct += (torch.max(outputs, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader)
        epoch_acc = correct / total
        if verbose:
            print(f"Train loss {epoch_loss}, Accuracy {epoch_acc}")

def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = net(inputs)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader)
    accuracy = correct / total
    return loss, accuracy


# Federated Learning

In [11]:
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

In [12]:
class FlowerClient(NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

In [13]:
def client_fn(context: Context) -> Client:
    """Create a Flower client representing a single organization."""

    input_dim = 5000  # (đã được đặt trong load_datasets max_features=5000)

    # Khởi tạo mô hình
    net = Net(input_dim).to(DEVICE)



    partition_id = context.node_config["partition-id"]
    trainloader, valloader, _ = load_datasets(partition_id=partition_id)

    return FlowerClient(net, trainloader, valloader).to_client()


# Create the ClientApp
client = ClientApp(client_fn=client_fn)

In [14]:
def weighted_average(metrics: List[Tuple[int, dict]]) -> dict:
    """
    Compute weighted average for metrics.
    
    Args:
        metrics: List of tuples (num_examples, metrics_dict).
    
    Returns:
        dict: Aggregated metrics with weighted averages.
    """
    # Ensure there are metrics to aggregate
    if not metrics:
        return {}

    # Initialize storage for weighted sums
    weighted_sums = {}
    total_examples = 0

    for num_examples, metric_dict in metrics:
        total_examples += num_examples
        for key, value in metric_dict.items():
            if key not in weighted_sums:
                weighted_sums[key] = 0
            weighted_sums[key] += num_examples * value

    # Compute weighted averages
    aggregated_metrics = {
        key: weighted_sums[key] / total_examples for key in weighted_sums
    }
    return aggregated_metrics


In [15]:
# Custom SaveModelStrategy implementation
class SaveModelStrategy(FedAvg):
    def aggregate_fit(
        self,
        server_round: int,
        results: list[tuple[ClientProxy, FitRes]],
        failures: list[Union[tuple[ClientProxy, FitRes], BaseException]],
    ) -> tuple[Optional[Parameters], dict[str, Scalar]]:

        # Call aggregate_fit from the base class (FedAvg)
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(
            server_round, results, failures
        )

        if aggregated_parameters is not None:
            # Convert `Parameters` to `list[np.ndarray]`
            aggregated_ndarrays = parameters_to_ndarrays(aggregated_parameters)

            # Save aggregated weights for each round
            print(f"Saving round {server_round} aggregated weights...")
            np.savez(f"round-{server_round}-weights.npz", *aggregated_ndarrays)

            # Save the final model at the end of training
            if server_round == 5:
                with open("DPA_model.pkl", "wb") as f:
                    pickle.dump(aggregated_ndarrays, f)
                print("Final model saved as 'DPA_model.pkl'")
                # Lưu dưới dạng PyTorch
                torch.save(aggregated_ndarrays, "DPA_model.pth")
                print("Final model saved as 'DPA_model.pth'")

        return aggregated_parameters, aggregated_metrics

In [16]:

# Define the server function
def server_fn(context: Context) -> ServerAppComponents:
    # Use the custom SaveModelStrategy
    strategy = SaveModelStrategy(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=10,
        min_evaluate_clients=5,
        min_available_clients=10,
        fit_metrics_aggregation_fn=weighted_average,
        evaluate_metrics_aggregation_fn=weighted_average,
    )

    # Configure the server for 5 rounds of training
    config = ServerConfig(num_rounds=5)

    return ServerAppComponents(strategy=strategy, config=config)


# Create a new server instance with the SaveModelStrategy
server = ServerApp(server_fn=server_fn)

In [17]:
# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

# When running on GPU, assign an entire GPU for each client
if DEVICE.type == "cuda":
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 1.0}}
    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

In [18]:
import os

# Disable oneDNN custom operations to avoid floating-point round-off errors
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

# Import TensorFlow and other required libraries
import tensorflow as tf

# Initialize TensorFlow
print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.18.0


In [19]:
# Run simulation
history = run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
    verbose_logging=True
)



DEBUG 2024-12-01 11:13:15,330:     Asyncio event loop already running.
DEBUG 2024-12-01 11:13:15,332:     Logger propagate set to False
DEBUG 2024-12-01 11:13:15,332:     Pre-registering run with id 7625326181220526061
DEBUG 2024-12-01 11:13:15,333:     Using InMemoryState
DEBUG 2024-12-01 11:13:15,333:     Using InMemoryState
INFO 2024-12-01 11:13:15,334:      Starting Flower ServerApp, config: num_rounds=5, no round_timeout
DEBUG 2024-12-01 11:13:15,336:     Buffer time delay: 5s
INFO 2024-12-01 11:13:15,337:      
INFO 2024-12-01 11:13:15,341:      [INIT]
INFO 2024-12-01 11:13:15,342:      Requesting initial parameters from one random client
DEBUG 2024-12-01 11:13:20,344:     Using InMemoryState
DEBUG 2024-12-01 11:13:20,345:     Registered 10 nodes
DEBUG 2024-12-01 11:13:20,345:     Supported backends: ['ray']
DEBUG 2024-12-01 11:13:20,347:     Initialising: RayBackend
DEBUG 2024-12-01 11:13:20,359:     Backend config: {'client_resources': {'num_cpus': 1, 'num_gpus': 0.0}, 'init_ar

(ClientAppActor pid=20136) Loading dataset for partition ID: 1
(ClientAppActor pid=20136) Client 1,3,5 is applying poisoning attack.


INFO 2024-12-01 11:13:53,306:      Received initial parameters from one random client
INFO 2024-12-01 11:13:53,308:      Starting evaluation of initial global parameters
INFO 2024-12-01 11:13:53,308:      Evaluation returned no results (`None`)
INFO 2024-12-01 11:13:53,309:      
INFO 2024-12-01 11:13:53,309:      [ROUND 1]
INFO 2024-12-01 11:13:53,310:      configure_fit: strategy sampled 10 clients (out of 10)


(ClientAppActor pid=20136) Partition 1: Train 502, Val 56, Test 558
(ClientAppActor pid=20136) Loading dataset for partition ID: 3
(ClientAppActor pid=20136) Client 1,3,5 is applying poisoning attack.
(ClientAppActor pid=20136) Partition 3: Train 501, Val 56, Test 557
(ClientAppActor pid=28884) Loading dataset for partition ID: 7
(ClientAppActor pid=12304) Loading dataset for partition ID: 0
(ClientAppActor pid=28884) Partition 7: Train 501, Val 56, Test 557
(ClientAppActor pid=18816) Client 1,3,5 is applying poisoning attack.


INFO 2024-12-01 11:14:10,280:      aggregate_fit: received 10 results and 0 failures
INFO 2024-12-01 11:14:10,326:      configure_evaluate: strategy sampled 5 clients (out of 10)


...

INFO 2024-12-01 11:14:21,366:      aggregate_evaluate: received 5 results and 0 failures
INFO 2024-12-01 11:14:21,367:      
INFO 2024-12-01 11:14:21,367:      [ROUND 5]
INFO 2024-12-01 11:14:21,367:      configure_fit: strategy sampled 10 clients (out of 10)
INFO 2024-12-01 11:14:23,912:      aggregate_fit: received 10 results and 0 failures
INFO 2024-12-01 11:14:24,020:      configure_evaluate: strategy sampled 5 clients (out of 10)


Saving round 5 aggregated weights...
Final model saved as 'DPA_model.pkl'
Final model saved as 'DPA_model.pth'
(ClientAppActor pid=33752) Loading dataset for partition ID: 1 [repeated 30x across cluster]
(ClientAppActor pid=33752) Partition 3: Train 501, Val 56, Test 557 [repeated 30x across cluster]
(ClientAppActor pid=33752) Client 1,3,5 is applying poisoning attack. [repeated 9x across cluster]


INFO 2024-12-01 11:14:24,653:      aggregate_evaluate: received 5 results and 0 failures
INFO 2024-12-01 11:14:24,658:      
INFO 2024-12-01 11:14:24,659:      [SUMMARY]
INFO 2024-12-01 11:14:24,659:      Run finished 5 round(s) in 31.35s
INFO 2024-12-01 11:14:24,661:      	History (loss, distributed):
INFO 2024-12-01 11:14:24,662:      		round 1: 0.6287896305322647
INFO 2024-12-01 11:14:24,662:      		round 2: 0.6946644857525826
INFO 2024-12-01 11:14:24,664:      		round 3: 0.5380954176187516
INFO 2024-12-01 11:14:24,665:      		round 4: 0.777078415453434
INFO 2024-12-01 11:14:24,665:      		round 5: 0.9981233939528465
INFO 2024-12-01 11:14:24,666:      	History (metrics, distributed, evaluate):
INFO 2024-12-01 11:14:24,666:      	{'accuracy': [(1, 0.7285714285714284),
INFO 2024-12-01 11:14:24,667:      	              (2, 0.5857142857142857),
INFO 2024-12-01 11:14:24,668:      	              (3, 0.7464285714285714),
INFO 2024-12-01 11:14:24,669:      	              (4, 0.5857142857142

(ClientAppActor pid=25532) Loading dataset for partition ID: 8 [repeated 4x across cluster]
(ClientAppActor pid=1052) Partition 3: Train 501, Val 56, Test 557 [repeated 5x across cluster]
(ClientAppActor pid=1052) Client 1,3,5 is applying poisoning attack. [repeated 3x across cluster]


DEBUG 2024-12-01 11:14:26,670:     Terminated RayBackend
DEBUG 2024-12-01 11:14:26,672:     Stopping Simulation Engine now.


In [21]:
def predict_sms_with_final_model(sms_text: str):
    """Predicts whether an SMS is spam or ham using the final trained model."""

    # Load the final model weights
    final_model_weights = torch.load("DPA_model.pth")

    # Initialize the model
    final_model = Net(input_dim).to(DEVICE)

    # Set the model parameters
    set_parameters(final_model, final_model_weights)

    # Vectorize the input SMS using the same vectorizer used during training
    vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
    vectorizer.fit(data["text"])  # Fit the vectorizer on the entire dataset
    sms_vector = vectorizer.transform([sms_text]).toarray().squeeze()
    sms_tensor = torch.tensor(sms_vector, dtype=torch.float32).to(DEVICE)

    # Make a prediction using the trained model
    with torch.no_grad():
        final_model.eval()  # Set the model to evaluation mode
        output = final_model(sms_tensor)
        _, predicted = torch.max(output.data, 0)  # Get the class with highest probability

    # Return the prediction (0 for ham, 1 for spam)
    return predicted.item()

# Example usage
input_sms = input("Enter an SMS message: ")
prediction = predict_sms_with_final_model(input_sms)

print("Input SMS: ", input_sms)
if prediction == 0:
    print("Prediction: Ham")
else:
    print("Prediction: Spam")

Input SMS:  Win a free iPhone today! Claim your reward now!
Prediction: Spam


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_24504\180352744.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  final_model_weights = torch.load("DPA_model.pth")
